In [1]:
# import library
import tensorflow as tf
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.utils import multi_gpu_model
from keras import metrics
from keras import losses
from keras import __version__
print('Using Keras version:', __version__, 'backend:', K.backend())
from keras.backend.tensorflow_backend import set_session

from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report # https://stackoverflow.com/questions/50065484/getting-precision-recall-and-f1-score-per-class-in-keras

from IPython.display import Javascript
import numpy as np
from distutils.version import LooseVersion as LV
from datetime import datetime
import csv
import sys
from os import listdir
from os.path import isfile, join
import pprint
import boto3
import pickle
import time
import os.path
import pickle
sys.path.append(os.getcwd())
from matplotlib import pyplot

import smtplib
from email.mime.text import MIMEText # simple MSG
from email.mime.multipart import MIMEMultipart # complex MSG
        
from link_aws_key import *
pp = pprint.PrettyPrinter(indent=4)

import warnings
warnings.filterwarnings("ignore")

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier


coins = {
    0: 'KRW',
    1: 'BTC',
    2: 'ETH',
    3: 'XRP',
    4: 'BCH',
    5: 'LTC',
    6: 'DASH',
    7: 'ETC'
}

# aws_client = boto3.client(
#     's3',
#     aws_access_key_id=LINK_AWSAccessKeyId,
#     aws_secret_access_key=LINK_AWSSecretKey
# )

bucket = "bithumb10"
cleanup_file_name = "coin_{0}_{1}_cleanup.csv"


#######################################################

def Load_Dataset_X(dir_path, time_unit, window_size, gap, margin_rate):
    key_name_X = "X_" + \
                    str(time_unit) + "_" + \
                    str(window_size) + "_" + \
                    str(gap) + "_" + \
                    str(margin_rate)
                    

    with open(dir_path + key_name_X + ".pickle", 'rb') as handle:
        b_x = pickle.load(handle)
    return b_x

def Load_Dataset_y(dir_path, time_unit, window_size, gap, margin_rate):
    key_name_y = "y_" + \
                    str(time_unit) + "_" + \
                    str(window_size) + "_" + \
                    str(gap) + "_" + \
                    str(margin_rate)
                    

    with open(dir_path + key_name_y + ".pickle", 'rb') as handle:
        b_y = pickle.load(handle)
    return b_y

def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def input_reshape(X_train_data, X_test_data, n_steps, n_coins, n_price):
    X_train_reshape = X_train_data.reshape(
        -1, 
        n_steps, 
        n_coins * n_price
    )
    X_test_reshape = X_test_data.reshape(
        -1, 
        n_steps, 
        n_coins * n_price
    )
    return X_train_reshape, X_test_reshape

def onehottify(x, n=None, dtype=np.int):
    """1-hot encode x with the max value n (computed from data if n is None)."""
    x = np.asarray(x)
    n = np.max(x) + 1 if n is None else n
    return np.eye(n, dtype=dtype)[x]

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

def Start_Model(pickle_load_dir_path, 
                pickle_result_dir_path, 
                idx_time_unit, 
                idx_window_size, 
                idx_gap, 
                idx_margin_rate, 
                epochs, 
                MODEL, 
                _GPU, 
                n_jobs,
                cv,
                dataset_scale,
                machine):
    X = {}
    y = {}

    key_name_X = "X_"
    key_name_y = "y_"

    key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
    key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)

    # remove [:10000], when real training
    X = Load_Dataset_X(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate)[0][:dataset_scale]
    y = Load_Dataset_y(pickle_load_dir_path, idx_time_unit, idx_window_size, idx_gap, idx_margin_rate)[1][:dataset_scale]
    
    y_single = {}
#     print("[INFO] y : {}".format(y))
#     y = np.asarray(y[0])
#     print("[INFO] y.shape : {}".format(y.shape))
#     print("[INFO] y : {}".format(y))
    y_single['BTC'] = y[:, 1]
    y_single['ETH'] = y[:, 2]
    y_single['XRP'] = y[:, 3]
    y_single['BCH'] = y[:, 4]
    y_single['LTC'] = y[:, 5]
    y_single['DASH'] = y[:, 6]
    y_single['ETC'] = y[:, 7]

    coin_list2 = ["BTC", "ETH", "XRP", "BCH", "LTC", "DASH", "ETC"]


    for coin in coin_list2:
        if (os.path.isfile(pickle_result_dir_path + \
                          MODEL + "_" + \
                          coin + "_" + \
                          str(idx_time_unit) + "_" + \
                          str(idx_window_size) + "_" + \
                          str(idx_gap) + "_" + \
                          str(idx_margin_rate) + \
                          "_result.pickle")) is True:
            print(MODEL + "_" + \
                  coin + "_" + \
                  str(idx_time_unit) + "_" + \
                  str(idx_window_size) + "_" + \
                  str(idx_gap) + "_" + \
                  str(idx_margin_rate) + \
                  "_result.pickle FILE ALREADY EXIST.")
            continue
        else:
            y2 = onehottify(y_single[coin], n=2)

            X_train, X_test, y_train, y_test = train_test_split(X, 
                                                                y2, 
                                                                test_size=0.1, 
                                                                random_state=42)
            print("[INFO] X_train.shape : {}".format(X_train.shape))
            print("[INFO] y_train.shape : {}".format(y_train.shape))
            print("[INFO] X_test.shape : {}".format(X_test.shape))
            print("[INFO] y_test.shape : {}".format(y_test.shape))
            print()

            n_coins = 8
            n_price = 4
            n_steps = idx_window_size 

            X_train_2 = X_train.transpose([0, 2, 1, 3])
            X_test_2 = X_test.transpose([0, 2, 1, 3])
            print("[INFO] X_train_2.shape: {}".format(X_train_2.shape))
            print("[INFO] X_test_2.shape: {}".format(X_test_2.shape))
            print()
            
            X_train_3 = X_train_2.reshape([X_train.shape[0], n_steps, n_coins * n_price])
            X_test_3 = X_test_2.reshape([X_test.shape[0], n_steps, n_coins * n_price])
            print("[INFO] X_train_3.shape: {}".format(X_train_3.shape))
            print("[INFO] X_test_3.shape: {}".format(X_test_3.shape))
            print()

            X_train_reshape = X_train_2.reshape([X_train.shape[0], n_steps*n_coins * n_price])
            X_test_reshape = X_test_2.reshape([X_test.shape[0], n_steps*n_coins * n_price])
            print("[INFO] X_train_reshape.shape: {}".format(X_train_reshape.shape))
            print("[INFO] X_test_reshape.shape: {}".format(X_test_reshape.shape))
            print()
            
            param_grid = {'window_size' : [idx_window_size], 
                          'n_state_units': [16, 32, 64],
                          'activation': ['tanh', 'sigmoid', 'relu'], 
                          'optimizer': ['rmsprop', 'Adam', 'Adagrad', 'SGD']}

            scaler = MinMaxScaler()
            scaler.fit(X_train_reshape)
            
            X_train_scaled = scaler.transform(X_train_reshape)
            X_test_scaled = scaler.transform(X_test_reshape)

            X_train_scaled = X_train_scaled.reshape(-1, 
                                                    n_steps, 
                                                    n_coins * n_price)
            X_test_scaled = X_test_scaled.reshape(-1, 
                                                  n_steps, 
                                                  n_coins * n_price)

            if _GPU == True:
                model = KerasClassifier(build_fn=create_model_LSTM, 
                                        epochs=epochs, 
    #                                     batch_size=100, 
                                        verbose=True)
                
            elif _GPU == False:
                model = KerasClassifier(build_fn=create_model_LSTM_non_GPU, 
                                        epochs=epochs, 
                                        batch_size=10, 
                                        verbose=True)

            grid = GridSearchCV(estimator=model, 
                                cv=cv, 
                                n_jobs=n_jobs, # test
                                param_grid=param_grid,
                                verbose=1)

            
            X_train_scaled, X_test_scaled = input_reshape(X_train_scaled,  
                                                          X_test_scaled, 
                                                          n_steps, 
                                                          n_coins, 
                                                          n_price)
                      
            
            print()
            print()
            print("----------------------")
            print("<"+MODEL+">")
            print("----------------------")
            print("__"+coin+"__" + \
                    "time unit: "+str(idx_time_unit) + "  |  " + \
                    "window_size :"+str(idx_window_size) + "  |  " + \
                    "gap :"+str(idx_gap) + "  |  " + \
                    "margin_rate :"+str(idx_margin_rate) + \
                    "  started.")

            grid_result = grid.fit(X_train_scaled, 
                                   y_train, 
                                   validation_data=(X_test_scaled,
                                                    y_test))

            print("----------------------")
            print("grid_result.score(X_test_scaled, y_test): ",grid_result.score(X_test_scaled, y_test))

            evaluate_result = {}

            test_score = grid_result.score(X_test_scaled, y_test)
            evaluate_result[MODEL + "_" + \
                          coin + "_" + \
                          str(idx_time_unit) + "_" + \
                          str(idx_window_size) + "_" + \
                          str(idx_gap) + "_" + \
                          str(idx_margin_rate)] = {"MODEL":MODEL,\
                                            "Cryptocurrency":coin, \
                                            "Score":grid_result.cv_results_['mean_test_score'], \
                                            "Params":grid_result.cv_results_['params'],\
                                            "test_score":test_score} 
        #     print()
        #     print("evaluate result dict: ", evaluate_result)
        #     print()

            # summarize results
            print()
            print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
            print()
            # for checking pickle file exist
            print("---pickle saving..")
            
            
            X = {}
            y = {}
            key_name_X = "X_"
            key_name_y = "y_"

            key_name_X += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
            key_name_y += str(idx_time_unit) + "_" + str(idx_window_size) + "_" + str(idx_gap) + "_" + str(idx_margin_rate)
            if (os.path.isfile(pickle_result_dir_path + \
                              MODEL + "_" + \
                              coin + "_" + \
                              str(idx_time_unit) + "_" + \
                              str(idx_window_size) + "_" + \
                              str(idx_gap) + "_" + \
                              str(idx_margin_rate) + \
                              "_result.pickle")) is not True:
                with open(pickle_result_dir_path + \
                          MODEL + "_" + \
                          coin + "_" + \
                          str(idx_time_unit) + "_" + \
                          str(idx_window_size) + "_" + \
                          str(idx_gap) + "_" + \
                          str(idx_margin_rate) + \
                          "_result.pickle", 'wb') as handle:
                    pickle.dump(evaluate_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

                    
            else:
                print("Already exist the file: ", pickle_result_dir_path + \
                                                  "_test_" + \
                                                  MODEL + "_" + \
                                                  "BTC" + "_" + \
                                                  str(idx_time_unit) + "_" + \
                                                  str(idx_window_size) + "_" + \
                                                  str(idx_gap) + "_" + \
                                                  str(idx_margin_rate) + \
                                                  "_result.pickle")

            
        #     for mean, stdev, param in zip(means, stds, params):
        #         print("%f (%f) with: %r" % (mean, stdev, param))
        #     print()
            key_name_X = "X_"
            key_name_y = "y_"


            return evaluate_result
        
def create_model_LSTM(window_size, n_state_units=32, activation='softmax', optimizer='adam', init='glorot_uniform', dropout_rate=0.0, neurons=2):
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        LSTM( n_state_units, 
              input_shape=(window_size, 32),
              use_bias=True, 
              activation='tanh',
              kernel_initializer='glorot_uniform', 
              recurrent_initializer='orthogonal', 
              bias_initializer='zeros', 
              dropout=0.0,
              recurrent_dropout=0.0))
    
    model.add(Dense(units=neurons))
    model.add(Dropout(dropout_rate))
#     model.add(Dense(units=2))
    model = multi_gpu_model(model, gpus=2)

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model


def create_model_LSTM_non_GPU(window_size, 
                              n_state_units=32, 
                              activation='softmax', 
                              optimizer='adam'):
    K.clear_session()
    model = Sequential()
    model.add(# if문을 통해 여러 RNN모델 쓸 수 있도록 하기, SimpleRNN외에 다른 RNN모델 찾아보기
        LSTM(units=n_state_units, 
             input_shape=(window_size, 32),
             use_bias=True))
    
    model.add(Dense(units=2))
#     model.add(Dropout(dropout_rate))

    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=optimizer, 
                  metrics=["accuracy", f1_score])

    return model

def start(machine, 
          _GPU,
          n_jobs, 
          MODEL, 
          idx_time_unit, 
          idx_window_size, 
          idx_gap, 
          idx_margin_rate, 
          cv,
          dataset_scale,
          epochs):
    
    '''
        [ATTENTION] In create_model METHOD part, need to set appropriate about GPU
        
        LINK01 -> GPU OFF
        MSI -> GPU OFF
        SLAVE04 -> GPU ON
        SLAVE05 -> GPU ON
    ''' 
    

    start_time = time.time()
    evaluate_result = Start_Model(pickle_load_dir_path = "../_dataset/RNN_coin/",  
                pickle_result_dir_path = "./evaluate_result/", 
                idx_time_unit=idx_time_unit,
                idx_window_size=idx_window_size, 
                idx_gap=idx_gap, 
                idx_margin_rate=idx_margin_rate, 
                epochs=epochs, 
                MODEL=MODEL, 
                _GPU=_GPU,
                n_jobs=n_jobs,
                cv=cv,
                dataset_scale=dataset_scale,
                machine=machine)
    print("DEBUG")
    end_time = time.time()
    print()
    print("TIME: ", end_time-start_time)
    return evaluate_result

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Using Keras version: 2.2.2 backend: tensorflow


In [ ]:
model_info = {}
model_info["window_size_standard"] = {"MODEL_list":["LSTM"],
                                      "time_unit":[10], 
                                      "window_size":[10, 25, 50, 75, 100],
                                      "gap":[1], 
                                      "margin_rate":[0.1],
                                      "lr":[0.01, 0.001]}

Machine = "window_size_standard"
# If Machine is "test" then, _TEST argument must be True
test = True
_GPU = False
n_jobs = -1
epochs = 1
cv = 2 # at least 2
dataset_scale = 1000
evaluate_result_dict = {}
coin_list = ["BTC", "ETH", "XRP", "BCH", "LTC", "DASH", "ETC"]

for model in model_info[Machine]["MODEL_list"]:
    for coin in coin_list:
        for idx_time_unit in model_info[Machine]["time_unit"]:
            for idx_window_size in model_info[Machine]["window_size"]:
                for idx_gap in model_info[Machine]["gap"]:
                    for idx_margin_rate in model_info[Machine]["margin_rate"]:
                        key_pickleFileName = model + '_' + coin + '_' + str(idx_time_unit) + '_' + str(idx_window_size) + '_' + str(idx_gap) + '_' + str(idx_margin_rate)
#                         print(key_pickleFileName)
                        try: 
                            evaluate_result_dict[key_pickleFileName] = start(machine=Machine, 
                                                                          _GPU=_GPU, 
                                                                          n_jobs=1,
                                                                          MODEL = model, 
                                                                          idx_time_unit=idx_time_unit, 
                                                                          idx_window_size=idx_window_size, 
                                                                          idx_gap=idx_gap, 
                                                                          idx_margin_rate=idx_margin_rate,
                                                                          cv=cv,
                                                                          dataset_scale=dataset_scale,
                                                                          epochs=epochs)

                            print("{:.2f}".format(evaluate_result_dict[key_pickleFileName]['test_score']))
                            
                        except KeyError:
                            print("[INFO] Appropriate value of {:4s} is not exist.".format(coin))
                            continue